In [1]:
import pandas as pd
#A random file with trips
df = pd.read_csv('sensorlogger_StopsOpenData.csv')
df

,Unnamed: 0,datetime,lng,lat,uid,leaving_datetime
0,0,2024-05-07 10:39:47,-8.401850,40.190362,146,2024-05-07 10:58:50
1,1,2024-05-07 11:05:39,-8.409373,40.198672,146,2024-05-07 16:49:52
2,2,2024-05-07 16:49:52,-8.416462,40.186296,146,2024-05-07 18:28:10
3,3,2024-05-07 18:29:49,-8.429089,40.197293,146,2024-05-07 19:13:21
4,4,2024-05-07 19:13:21,-8.425924,40.202361,146,2024-05-07 19:37:31
5,5,2024-05-07 19:37:31,-8.432195,40.204944,146,2024-05-07 21:34:39
6,6,2024-05-07 21:34:39,-8.430780,40.200921,146,2024-05-07 22:00:13
7,7,2024-05-07 22:00:13,-8.409364,40.198674,146,2024-05-08 14:00:22
8,8,2024-05-08 14:00:22,-8.414995,40.186314,146,2024-05-08 19:45:45
9,9,2024-05-08 19:45:45,-8.408615,40.194200,146,2024-05-08 20:17:25


In [2]:
#Obtain the column names, number of instances and datatype of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        20 non-null     int64  
 1   datetime          20 non-null     object 
 2   lng               20 non-null     float64
 3   lat               20 non-null     float64
 4   uid               20 non-null     int64  
 5   leaving_datetime  20 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.1+ KB


In [9]:
import requests
from geopy.distance import geodesic

# Replace with your Foursquare API key
FOURSQUARE_API_KEY = 'fsq3xjD2KIb7kY5oJGt5/4/nYydBjx0oz1ofCaLjE0WBRYY='
SEARCH_RADIUS = 10  # Radius in meters

def get_nearest_poi_category(lat, lon):
    url = 'https://api.foursquare.com/v3/places/search'
    params = {
        'll': f'{lat},{lon}',
        'radius': SEARCH_RADIUS,
        'limit': 1  # Only need the nearest POI
    }
    headers = {
        'Authorization': FOURSQUARE_API_KEY
    }

    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['categories'][0]['name']
        else:
            return 'No POI within 10 meters'
    else:
        return 'API request failed'

# Apply the function to each row in the DataFrame
df['poi_category'] = df.apply(lambda row: get_nearest_poi_category(row['lat'], row['lng']), axis=1)

print(df)

    Unnamed: 0             datetime       lng        lat  uid  \
0            0  2024-05-07 10:39:47 -8.401850  40.190362  146   
1            1  2024-05-07 11:05:39 -8.409373  40.198672  146   
2            2  2024-05-07 16:49:52 -8.416462  40.186296  146   
3            3  2024-05-07 18:29:49 -8.429089  40.197293  146   
4            4  2024-05-07 19:13:21 -8.425924  40.202361  146   
5            5  2024-05-07 19:37:31 -8.432195  40.204944  146   
6            6  2024-05-07 21:34:39 -8.430780  40.200921  146   
7            7  2024-05-07 22:00:13 -8.409364  40.198674  146   
8            8  2024-05-08 14:00:22 -8.414995  40.186314  146   
9            9  2024-05-08 19:45:45 -8.408615  40.194200  146   
10          10  2024-05-14 10:46:40 -8.405773  40.197228  217   
11          11  2024-05-14 19:06:40 -8.412925  40.192630  217   
12          12  2024-05-14 21:53:20 -8.405745  40.197138  217   
13          13  2024-05-27 10:12:12 -8.405779  40.197210  222   
14          14  2024-05-2

In [10]:
df.to_csv('sensorlogger_StopsOpenDataFoursquare.csv')